# 6章 デプロイ

## 6.1 継続的インテグレーションとは

* CIの主な目的はすべてを互いに常に同期させること
  * 実際には新たにチェックインされたコードが既存コードと適切に統合（インテグレーション）されるようにすること


* 理想的には一度だけのビルドで、そのバージョンの他のコードのすべてのデプロイで再利用したい


* CIの利点
  * コードの品質に関して高速なフィードバック
  * バイナリ作成物の作成を自動化
  * 必要に応じて成果物を再作成可能
  * 成果物からコードへのある程度のトレーサビリティ

## 6.1.1 実際にCIを行っているか

* CIを行っていると言っているにもかかわらず実際には全く行っていない多くのチーム
  * CIツールの利用とCIのプラクティスを混同している


* CIとは何かを本当に理解しているかを調べるための3つの質問 (Jez Humble)
  * 1日に一度はメインラインにチェックインしていますか。
    * メインブランチに頻繁にマージすること
  * 変更を検証するテストスイートがありますか。
  * ビルドが壊れたときに、それを修正するのがチームの最優先事項でしょうか。
    * 壊れたビルドの修正とは関係ない変更を増やしていくと、ビルドを修正するためにかかる時間が増えていく

## 6.2 継続的インテグレーションとマイクロサービスへのマッピング

* CIビルドを個々のマイクロサービスにどのようにマッピングするか
  * 残りのサービスとは独立して、1つのサービスに変更を行いそれをデプロイできるようにしたい


* 勧められないが最も簡単な方法はすべてをひとまとめにすること
  * 全コードを格納する巨大なリポジトリと1つのビルド
  * 同時リリースを行う場合にはうまく機能する
    * 同時リリースは一般的には絶対避けるべき
    * プロジェクトの初期段階など短期的には理にかなっていることもある
  * 欠点
    * 1つのサービスを変更すると他のサービスもすべて検証されてビルドされる
    * 関係のないビルドのために時間が大幅にかかる
    * どれをデプロイすべきか判断しにくい
    * ビルドを壊してしまったときに他のサービスの変更ができなくなる
    * ビルドの責任を誰が負うか
  * この方法のバリエーションは複数のCIビルドをソースツリーの一部に対応付ける方法
    * 複数サービスのソースコードを一度にチェックインする習慣がとてもつきやすくなり、サービスを結合する変更にも陥りやすくなる (?)


* 代替手段: マイクロサービスごとに1つのCIビルドを持つこと

## 6.3 ビルドパイプラインと継続的デリバリ

* ビルドには複数のステージがあると便利
  * 多数の高速でスコープの小さいテストと少数の低速でスコープの大きいテスト
  * 高速のテストで失敗したら低速のテストをやる意味はあまりない
  * ビルドパイプラインを作成する


* 継続的デリバリ
  * 手動と自動の両方でソフトウェアが通過しなければいけないあらゆるステージをモデル化
  * CIツールをハックし拡張してCDを実行するのではなく、CDを第一級の概念として採用しているツールが必要
  * 手動のユーザ受け入れテスト(UAT: User Acceptance Testing)もCDのモデルの中に組み込む
    * 手動でテストを行ったあと、後続のステージに進むトリガーを手動で設定
  * ソフトウェアの本番環境までの経路全体をモデル化すると、 ... リリースにかかる時間も大幅に短縮できます。

## 6.3.1 避けられない例外

* 基本的には「ビルドごとに1つのマイクロサービス」の手法を目指すべき
* 例外
  * 新規のプロジェクトではすべてのサービスを1つのビルドにすることは理にかなっている
    * その場合は当然すべてのサービスが一括リリースになる
  * サービス境界を割り出すために大きな変動がある可能性が高い
  * サービスAPIが安定したらビルドを分割する移行ステージが必要

## 6.4 プラットフォーム固有の成果物

* 成果物の作成とインストールをサポートするツール
  * Rubyのgem、JavaのJARファイル、WARファイル、PythonのEgg
* 構成管理ツール
  * Puppet, Chef, Ansible
  * 成果物を使って実行するために必要な他のツールをインストールして構成するもの
  * RubyやPythonのアプリケーションではApache, nginx内で動作するプロセスマネージャが必要


* さまざまな技術を組み合わせて使うときはデプロイが困難になる
  * 構成管理ツールを使った自動化がデプロイ機構の違いを隠すのに役立つ

## 6.5 OS成果物

* OS固有の成果物
  * Red Hat, CentOSのRPM、Ubuntuのdebパッケージ、WindowsのMSI
  * デプロイの観点から見て基盤となる技術を気に掛けなくてよい
  * Linuxプラットフォームではパッケージの依存関係を定義でき、自動的にインストール可能


* パッケージ作成が困難
  * LinuxではFPMパッケージマネージャツール
  * WindowsではMSIインストーラは機能的に不十分
    * Chocolatey NuGetがLinuxのパッケージマネージャに似ている


* 複数の異なるOSへのデプロイが困難

## 6.6 カスタムイメージ

* 自動構成管理システム(Puppet, Chef, Ansible)は実行するのに時間がかかる
  * マシンのプロビジョニングに数分、Javaアプリケーションを実行するためにOracle JVMをインストールに数分
  * さらにOS統計情報を取得するためにcollectd、ログ集約にLogstash、監視のためにNagios
  * 必要なものが増え、依存関係が増えるごとにかかる時間は長くなっていく
  * 高速なフィードバックを提供するという観点で深刻な問題となる


* システムがゼロダウンタイムデプロイを許していない場合 (?)
  * ダウンタイムがあってもよいということ?


* 仮想マシンイメージの作成
  * 起動時間の短縮
  * 依存関係を含めた仮想マシンイメージを作成しておく（カスタムイメージ）
  * カスタムイメージからインスタンスを起動し、サービスの最新バージョンをインストールするだけ
  * あらかじめイメージに含めておく依存関係が変更にならない限り、同じカスタムイメージを使い続けることができる


* カスタムイメージの欠点
  * イメージの作成に時間がかかる
  * イメージのサイズが大きくなる（コンテナ技術によって一部回避可能）


* イメージを構築するために必要なツールがプラットフォームごとに異なるという課題
  * VMwareイメージ、AWS AMI、Vagrantイメージ、Rackspaceイメージの構築はそれぞれ異なる
  * Packerは同じ構成でさまざまなプラットフォーム向けのイメージを作成できる

## 6.6.1 成果物としてのイメージ

* 依存関係だけではなくサービスもイメージに含める